In [1]:
import json
import numpy as np
import cs_vqe.circuit as cs_circ
import utils.plotting_tools as plot
import utils.qonversion_tools as qonvert
import utils.linalg_tools as la
import utils.bit_tools as bit
import utils.cs_vqe_tools_original as cs_tools
from qat.lang.AQASM import Program, X, H, S, CNOT, RZ
from qat.core.variables import Variable
from qat.core import Observable, Term
from matplotlib import pyplot as plt
from qat.plugins import ScipyMinimizePlugin
from qat.qpus import Stabs, LinAlg, MPS, Feynman
import random

Qiskit Nature not installed


In [2]:
with open('data/model_data.json', 'r') as json_file:
    model_data = json.load(json_file)

for mol in model_data.keys():
    print(mol, model_data[mol]['chem_acc_num_q'], len(model_data[mol]['uccsd']))

HeH+_6-311G_SINGLET 9 70
H2O_STO-3G_SINGLET 7 900
BeH+_STO-3G_SINGLET 6 292
LiH_STO-3G_SINGLET 4 388
CH+_STO-3G_SINGLET 6 160
HF_STO-3G_SINGLET 4 96
B+_STO-3G_SINGLET 3 176
OH+_STO-3G_SINGLET 5 104
CH2_STO-3G_SINGLET 8 624
BeH2_STO-3G_SINGLET 7 1276
Be_STO-3G_SINGLET 3 160
C_STO-3G_SINGLET 4 120
NH_STO-3G_SINGLET 6 108
BH_STO-3G_SINGLET 6 184
NH2+_STO-3G_SINGLET 8 564
BH2+_STO-3G_SINGLET 8 1152


In [3]:
speciesname = "HF_STO-3G_SINGLET"
mol_model    = model_data[speciesname]
uccsd = mol_model['uccsd']

In [4]:
mol_circ = cs_circ.cs_vqe_circuit(model_data=mol_model)

num_sim_q = mol_circ.chem_acc_num_q
print('Number of terms in the UCCSD ansatz:', len(uccsd))
print('%s reaches chemical accuracy for %i-qubit CS-VQE' % (speciesname, num_sim_q))
#mol_circ.plot_cs_vqe_errors()

Number of terms in the UCCSD ansatz: 96
HF_STO-3G_SINGLET reaches chemical accuracy for 4-qubit CS-VQE


In [5]:
def build_qlm_circuit(instructions, num_sim_q, num_params):
    gate_dict = {'X':X,'H':H,'S':S,'SDG':S.dag(),'RZ':RZ,'CX':CNOT}
    prog = Program()
    qbits_reg = prog.qalloc(num_sim_q)
    params = [prog.new_var(float, "\\P{}".format(i)) for i in range(num_params)]

    for gate, var_num, q_pos in instructions:
        q_pos = [num_sim_q-p-1 for p in q_pos] #qubit ordering is reversed compared with Qiskit
        if gate == 'RZ':
            prog.apply(RZ(params[var_num[0]]), qbits_reg[q_pos[0]])
        elif gate == 'CX':
            prog.apply(CNOT, qbits_reg[q_pos[0]], qbits_reg[q_pos[1]])
        else:
            prog.apply(gate_dict[gate], qbits_reg[q_pos[0]])

    qc = prog.to_circ()
    
    return qc


def plot_qlm_result(result):
    nfev = len(eval(result.meta_data['optimization_trace']))
    energy=eval(result.meta_data['optimization_trace'])

    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot([np.log10(abs(e-mol_circ.truegs)) for e in energy], label='CS-VQE convergence', color = "black")
    ax.hlines(np.log10(0.0016), 0, nfev, color='green', label='Chemical accuracy', ls='--')
    ax.hlines(np.log10(mol_circ.noncon-mol_circ.truegs), 0, nfev, color='red', label='Noncontextual ground state energy', ls='--')
    #plt.hlines(true_gs+0.0016, 0, nfev, color='pink')
    ax.set_title('{}: {}-qubit CS-VQE simulation'.format(speciesname, num_sim_q))
    ax.set_xlabel("Optimization count")
    ax.set_ylabel("Logarithmic error [log(Ha)]")
    ax.legend()
    return fig

In [6]:
theta0 = mol_circ.init_param
linalg_qpu = LinAlg(use_GPU=False)
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                                      tol=1e-3,
                                      options={"maxiter": 10000})
qpu = optimizer_scipy | linalg_qpu

In [ ]:
print('Noncon error:', mol_circ.noncon-mol_circ.truegs, '\n')
chemacc=0.0016
current_error=1
final_ops = ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI', 'YYIIIIXY', 'IYIIZZZX', 'XXZYZZZZ'] #HF progress
while current_error > chemacc:
    pauli_errors = []
    lost = []
    for pauli in uccsd.keys():
        if pauli not in final_ops:
            anz_ops = final_ops + [pauli] #random.choices(list(uccsd.keys()), k=30)
            #print('Ansatz operators:', anz_ops)
            anz = {op:uccsd[op] for op in anz_ops}
            ham_dict, num_sim_q, num_params, instructions = mol_circ.qlm_circuit(anz, num_sim_q).values()
            pauli_terms = [Term(coeff, op, list(range(num_sim_q))) for op, coeff in ham_dict.items()]
            hamiltonian = Observable(num_sim_q,pauli_terms=pauli_terms)

            qc = build_qlm_circuit(instructions, num_sim_q, num_params)
            try:
                job = qc.to_job(job_type="OBS", observable=hamiltonian)

                result = qpu.submit(job)
                error = result.value-mol_circ.truegs
                pauli_errors.append((pauli, error))
                #print("Minimum energy =", result.value, '| error =', error, '| Chemical accuracy?', error<0.0016)
                #print("Optimal angles =", result.meta_data["parameters"], '\n')


                if error<0.0016:
                    #final_ops.append(pauli)
                    plot_qlm_result(result)
                    break
            except:
                print('<!> QLM error \n')
                lost.append(pauli)

    pauli_out, current_error = sorted(pauli_errors, key=lambda x:x[1])[0]
    final_ops.append(pauli_out)
    
    print('Ansatz terms:', final_ops, '| error =', current_error, '\n')
    if lost!=[]:
        print('QLM error on terms:', lost)
    
print(final_ops)

Noncon error: 0.03229070928263411 

Ansatz terms: ['XXIIXYII'] | error = 0.006941272326372427 

<!> QLM error 

Ansatz terms: ['XXIIXYII', 'XYIIIIXX'] | error = 0.005137061847477753 

QLM error on terms: ['ZYIIIIII']
Ansatz terms: ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI'] | error = 0.0050541033977395955 

Ansatz terms: ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI', 'YYIIIIXY'] | error = 0.0043815379421943135 

Ansatz terms: ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI', 'YYIIIIXY', 'IYIIZZZX'] | error = 0.0031341361540597745 

Ansatz terms: ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI', 'YYIIIIXY', 'IYIIZZZX', 'XXZYZZZZ'] | error = 0.0023499389624390687 

Ansatz terms: ['XXIIXYII', 'XYIIIIXX', 'XZIIZZYI', 'YYIIIIXY', 'IYIIZZZX', 'XXZYZZZZ', 'YXZZIZXX'] | error = 0.0025876543975869026 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

<!> QLM error 

In [ ]:
import numpy as np
from qat.dqs.hamiltonians import SpinHamiltonian

ham_rot = cs_tools.rotate_operator(mol_circ.ham_rotations, ham)
full_terms = [Term(coeff, op, list(range(num_qubits))) for op, coeff in ham_rot.items()]
ham_spin = SpinHamiltonian(nqbits=num_qubits, pauli_terms=full_terms)
ham_matrix = ham_spin.get_matrix()
eigvals, eigvecs = np.linalg.eigh(ham_matrix)
gs = sorted(list(zip(eigvals, eigvecs)), key=lambda x:x[0])[0]

print("Exact ground state energy: ", gs[0])
basis_states = [bit.int_to_bin(i, num_qubits) for i in range(2**num_qubits)]
sig_states = sorted(list(zip(abs(gs[1])**2, basis_states)), key=lambda x:-x[0])
print(sig_states[:5])

fig, ax = plt.subplots()
l1=ax.bar(basis_states, [abs(a)**2 for a in gs[1]])
l2=ax.set_xticklabels(basis_states, rotation=90)

In [ ]:
nonconstate = mol_circ.reference_state()
print(nonconstate)
reverse_rot = []
for angle, rot in mol_circ.ham_rotations[::-1]:
    if angle == 'pi/2':
        reverse_rot.append([-np.pi/2, rot])
    else:
        reverse_rot.append([-angle, rot])

anz={}
for amp, state in sig_states[:2]:
    differs=[index for index, b in enumerate(state) if nonconstate[index]!=b]
    #pot_ops=[]
    #for op in uccsd.keys():
    #    diff_bits = {op[i] for i in differs}
    #    same_bits = {op[i] for i in range(num_qubits) if i not in differs}
    #    if diff_bits in [{'X'}, {'Y'}, {'X', 'Y'}] and same_bits in [{'I'},{'Z'},{'I', 'Z'}]:
    #        pot_ops.append(op)
    blank_op = ['I' for i in range(num_qubits)]
    for i in differs:
        blank_op[i] = 'X'
    op = ''.join(blank_op)
    if set(op)!={'I'}:
        anz[op] = amp

out_anz = cs_tools.rotate_operator(reverse_rot, anz)
out_anz

In [ ]:
import random

#anz_ops = ['ZYXZI', 'YIIYX']#for C
#anz_ops = ['XIIYX', 'IYZYY'] #for B+
#anz_ops = ['ZYIXX', 'YIIXX', 'IIXXY', 'XIIXY', 'ZYIYX', 'IYZYX'] #for Be opt_angle = [0.2399988337483999, 0.4314907226264347, -0.08681710171587653, 0.10436738811510829, 0.0647405005029628, 0.1015767738121943, 0.1380455733703941, -0.1615323405241012, 0.07726029041157381, 0.1239271294076503] 
anz_ops = random.choices(list(uccsd.keys()), k=5)
#print(anz_ops)
anz = {op:uccsd[op] for op in anz_ops}
#anz=uccsd
ham_dict, num_sim_q, num_params, instructions = mol_circ.qlm_circuit(anz, num_sim_q).values()

In [ ]:
anz_ops

In [ ]:
cs_tools.rotate_operator(mol_circ.ham_rotations, anz)

In [ ]:
mol_dict = mol_circ.qlm_circuit(anz, num_sim_q)

In [ ]:
#data={}
#for k in mol_dict.keys():
#    data[str(k)] = mol_dict[k]
    
#with open("data/QLM_circuits/"+speciesname+"_UCCSD_CS-VQE_circuit.json", "w") as outfile: 
#    json.dump(data, outfile)

In [ ]:
pauli_terms = [Term(coeff, op, list(range(num_sim_q))) for op, coeff in ham_dict.items()]
hamiltonian = Observable(num_sim_q,
               pauli_terms=pauli_terms)
print("Hamiltonian:", hamiltonian)

In [ ]:
gate_dict = {'X':X,'H':H,'S':S,'SDG':S.dag(),'RZ':RZ,'CX':CNOT}
prog = Program()
qbits_reg = prog.qalloc(num_sim_q)
params = [prog.new_var(float, "\\P{}".format(i)) for i in range(num_params)]

for gate, var_num, q_pos in instructions:
    q_pos = [num_sim_q-p-1 for p in q_pos] #qubit ordering is reversed compared with Qiskit
    if gate == 'RZ':
        prog.apply(RZ(params[var_num[0]]), qbits_reg[q_pos[0]])
    elif gate == 'CX':
        prog.apply(CNOT, qbits_reg[q_pos[0]], qbits_reg[q_pos[1]])
    else:
        prog.apply(gate_dict[gate], qbits_reg[q_pos[0]])

qc = prog.to_circ()

In [ ]:

#from qat.plugins import SPSAMinimizePlugin
#from qat.plugins import PSOMinimizePlugin

theta0 = mol_circ.init_param
linalg_qpu = LinAlg(use_GPU=False)

#optimizer_pso = PSOMinimizePlugin(theta0, max_iter=50, swarm_number=10)
#optimizer_spsa = SPSAMinimizePlugin(theta0)
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                                      tol=1e-3,
                                      options={"maxiter": 2000})

qpu = optimizer_scipy | linalg_qpu

job = qc.to_job(job_type="OBS", observable=hamiltonian)

result = qpu.submit(job)

print("Minimum energy =", result.value, "... chemacc?", result.value-mol_circ.truegs<0.0016)
print("Optimal angles =", result.meta_data["parameters"])

In [ ]:
nfev = len(eval(result.meta_data['optimization_trace']))
energy=eval(result.meta_data['optimization_trace'])

fig, ax = plt.subplots(figsize=(10,6))
ax.plot([np.log10(abs(e-mol_circ.truegs)) for e in energy], label='CS-VQE convergence', color = "black")
ax.hlines(np.log10(0.0016), 0, nfev, color='green', label='Chemical accuracy', ls='--')
ax.hlines(np.log10(mol_circ.noncon-mol_circ.truegs), 0, nfev, color='red', label='Noncontextual ground state energy', ls='--')
#plt.hlines(true_gs+0.0016, 0, nfev, color='pink')
ax.set_title('{}: {}-qubit CS-VQE simulation'.format(speciesname, num_sim_q))
ax.set_xlabel("Optimization count")
ax.set_ylabel("Logarithmic error [log(Ha)]")
ax.legend()

In [ ]:
#fig.savefig('plots/'+speciesname+'_CS-VQE_test.png', dpi=300, bbox_inches='tight')

In [ ]:
#data={}
#for k in result.meta_data.keys():
#    data[str(k)] = result.meta_data[k]
#    
#with open("data/QLM_circuits/"+speciesname+"_CS-VQE_test.json", "w") as outfile: 
#    json.dump(data, outfile)